# AthenaK scaling on ALCF Polaris

2022-05-19

## Description of datasets

- GRMHD Torus v1: `weak_scaling.csv`, `strong_scaling_11_0.csv`, `strong_scaling.csv`
  - Many small MeshBlocks, 512x 32^3 per GPU
  - All outputs disabled
  - Ran for `time/tlim=100.0`. Default limit of 1e4 would take ~6 hrs on one GPU
  - Only problem configuration for which strong scaling tests were run.
  - Strong scaling looked better compared to abysmal weak scaling results, since it results in fewer tiny MeshBlocks per GPU as you scale up in a strong scaling test.
 
- GRMHD Torus v2: `weak_scaling_large_mbs.csv`
  - Fewer large MeshBlocks, 16x 128^3 per GPU
  - Otherwise, same parameters as before
  
- GRMHD (no-rad) linear wave: `weak_scaling_grmhd_linwave.csv`

- GRMHD radiation linear wave, level 2 spherical mesh: `weak_scaling_grrad_linwave_nlevel2.csv`
  - The number of angles is = 10*(level^2)+2
  - 42 angles

- GRMHD radiation linear wave, level 3 spherical mesh: `weak_scaling_grrad_linwave_nlevel3.csv`
  - 92 angles
  - Would not run without exhausting the 40.0 GiB of GPU memory 

## Notes

- Zsh is broken on compute nodes
- For sub-node scaling results (1, 2, or 3 GPUs), need to manually 
- Decent improvement moving from CUDA Toolkit 11.0 to 11.6. Compare 2x strong scaling CSV files.
- Needed commit from `compile_hotfix` branch to compile correctly with CUDA 11.6
- Seg-fault will occur at runtime unless `export MPICH_GPU_SUPPORT_ENABLED=1` is executed to enable GPUDirect/CUDA-aware Cray MPICH. Not needed at compile time 
- For all scaling tests, I would proportionally scale the `mesh/x3min, mesh/x3max, mesh/nx3` parameters so that the resolution remains fixed for all problems (and hence the timestep)

### Compilation instructions
Best to compile on compute node, so CMake can autodetect the A100s. Example for compiling and running on 1 node:
```
qsub -q run_next -I -l select=1:ncpus=64:ngpus=4,walltime=03:00:00 -j oe -S /bin/bash

module load craype-accel-nvidia80
export MPICH_GPU_SUPPORT_ENABLED=1

cd athenak
rm -rfd build
mkdir build; cd build
cmake -D Athena_ENABLE_MPI=ON -DKokkos_ENABLE_CUDA=On -DKokkos_ENABLE_CUDA_LAMBDA=On -DKokkos_ARCH_AMPERE80=On -DCMAKE_CXX_COMPILER=/home/felker/athenak/kokkos/bin/nvcc_wrapper ..
make -j 32

cd src
cp ~/athenak-scaling/*.sh ./

mpiexec -np 1 -ppn 4 -d 16 --cpu-bind depth -env OMP_NUM_THREADS=16 ./gpu_affinity.sh
```

- GR Torus requires `-DPROBLEM=gr_torus` in CMake command. Used `compile_hotfix` branch
- GRMHD (no radiation) linear wave requires `-DPROBLEM=rad_linear_wave` in CMake command. Used `scaling_grrad` branch
- GRMHD Radiation linear wave requires **no** `-DPROBLEM` option in CMake command. Used `scaling_grmhd` branch


### To do
- [ ] Get dedicated full machine reservation; compare scaling efficiency loss at >= 64 nodes
- [ ] Also retest after Slingshot 11 NIC upgrade in the fall
- [ ] Run 512 and/or 550 node jobs when the ~50 down nodes are brought back into service
- [ ] Explain the 7x efficiency loss from 1:8 GPUs when the 512x 32^3 MeshBlocks were used in the original GR torus problem setup

# Code

In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
import numpy as np

import pandas as pd

mpl.rcParams['figure.dpi'] = 300

from IPython.display import Image

import seaborn as sns


## Strong scaling, 1 to 16 nodes

In [ ]:
data = pd.read_csv('strong_scaling.csv')
data['Speedup'] = data['zone-cycles/cpu_second']/data['zone-cycles/cpu_second'][0]

In [ ]:
data

In [ ]:
data['zone-cycles/cpu_second'][0]

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['zone-cycles/cpu_second'],'-o')
#ax.set_xlabel('Number of MPI ranks = N_A100')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')

# ax.set_xscale('log', base=2)

ax.set_ylabel('Zone-cycles/second')
ax.axvline(x=4, color='0.8', alpha=0.8)


In [ ]:
fig.savefig("strong-scaling.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['Speedup'],'-o')
#ax.set_xlabel('Number of MPI ranks = N_A100')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Speedup')

ax.axvline(x=4, color='0.8', alpha=0.8)


In [ ]:
fig.savefig("strong-scaling-speedup.pdf")

## Weak scaling

In [ ]:
#data = pd.read_csv('weak_scaling.csv')
#data = pd.read_csv('weak_scaling_large_mbs.csv', comment='#')
# data = pd.read_csv('weak_scaling_grmhd_linwave.csv', comment='#')
# data = pd.read_csv('weak_scaling_grrad_linwave_nlevel2.csv', comment='#')
data = pd.read_csv('weak_scaling_grrad_linwave_nlevel3.csv', comment='#')

data['Scaled speedup'] = data['zone-cycles/cpu_second']/data['zone-cycles/cpu_second'][0]
data['Efficiency'] = data['cpu time used'][0]/data['cpu time used']

In [ ]:
data

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['cpu time used'],'-o')
#ax.set_xlabel('Number of MPI ranks = N_A100')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Wall time (s)')
ax.axvline(x=4, color='0.8', alpha=0.8)


In [ ]:
fig.savefig("weak-scaling-walltime.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['cpu time used'],'-o')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Wall time (s)')
ax.axvline(x=4, color='0.8', alpha=0.8)
ax.set_xscale('log', base=2)


In [ ]:
fig.savefig("weak-scaling-walltime-semilogy.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['Efficiency'],'-o')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Efficiency')
ax.axvline(x=4, color='0.8', alpha=0.8)

In [ ]:
fig.savefig("weak-scaling-efficiency.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['Efficiency'],'-o')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Efficiency')
ax.axvline(x=4, color='0.8', alpha=0.8)
ax.set_xscale('log', base=2)

In [ ]:
fig.savefig("weak-scaling-efficiency-semilogy.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['zone-cycles/cpu_second']/data['Num MPI ranks'],'-o')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Normalized performance [zone-cycles/second/GPU]')
ax.axvline(x=4, color='0.8', alpha=0.8)

#ax.set_xscale('log', base=2)

In [ ]:
fig.savefig("weak-scaling-normalized-performance.pdf")

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['Num MPI ranks'], data['zone-cycles/cpu_second']/data['Num MPI ranks'],'-o')
ax.set_xlabel(r'$N_{\mathrm{MPI}} = N_{\mathrm{A100}}$')
ax.set_ylabel('Normalized performance [zone-cycles/second/GPU]')
ax.axvline(x=4, color='0.8', alpha=0.8)

ax.set_xscale('log', base=2)

In [ ]:
fig.savefig("weak-scaling-normalized-performance-semilogy.pdf")

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(data['Num nodes'], data['zone-cycles/cpu_second']/data['Num MPI ranks'],'-o')
# ax.set_xlabel(r'$N_{\mathrm{Nodes}} = \frac{N_{\mathrm{A100}}}{4}$')
# ax.set_ylabel('Normalized performance [zone-cycles/second/GPU]')

In [ ]:
# fig.savefig("weak-scaling-normalized-performance-nodes.pdf")